# Which brand should you consider first for a used car on eBay Kleinanzeigen?

Cars are well-known as having a high depreciation rate. Therefore, purchasing used car can usually be economically more favorable for many people. *eBay Kleinanzeigen* is a classfied advertising section of the German eBay website. Here I will work with [a dataset](https://www.kaggle.com/orgesleka/used-cars-database/data) which was orinigally scraped from *eBay Kleinanzeigen*, for the used car sell information. My goals are to:
- identify which brand should a customer consider first, based on the popularity and the price. 
- generate a predictive model using linear regression to predict the prices of used cars with most popular brands.

## Data Cleaning

In [1]:
import pandas as pd
import numpy as np
autos = pd.read_csv('autos.csv', encoding = 'Latin-1')

I start by exploring the head of the dataframe:

In [2]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


It can be noticed that the dataset uses camelcase in column name intead of the conventional snakecase in Python. To make my later analysis easier, I am going to change all the column names into snakecase.

In [3]:
# Change column names
autos.columns = ['date_crawled','name','seller','offer_type','price','abtest','vehicle_type','registration_year','gearbox','power_ps','model','odometer','registration_month','fuel_type','brand','unrepaired_damage','ad_created','n_of_pictures','postal_code','last_seen']

Based on my objective, the most related cloumns are *price, power_ps, odometer*, and *brand*. Other factors that may have significant impact on prices are:
- *registration_year*: it indirectly reflects the milage
- *seller*: private sellers may have lower price as compared to commercial sellers 
- *vehicle_type*: different vehicle type has different price
- *model*: different model has different price
- *unrepaired_damage*: cars with not repaired damage may have a significantly lower price

Other factors should not impact sell price a lot and thus can be removed.

In [4]:
autos = autos[['seller','price','power_ps','odometer','registration_year','brand','vehicle_type','model','unrepaired_damage']]

Let's take a look at the description of the new dataframe and see if it can be further simplified.

In [5]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 9 columns):
seller               50000 non-null object
price                50000 non-null object
power_ps             50000 non-null int64
odometer             50000 non-null object
registration_year    50000 non-null int64
brand                50000 non-null object
vehicle_type         44905 non-null object
model                47242 non-null object
unrepaired_damage    40171 non-null object
dtypes: int64(2), object(7)
memory usage: 3.4+ MB


In [6]:
autos.describe(include='all')

,seller,price,power_ps,odometer,registration_year,brand,vehicle_type,model,unrepaired_damage
count,50000,50000,50000.000000,50000,50000.000000,50000,44905,47242,40171
unique,2,2357,NaN,13,NaN,40,8,245,2
top,privat,$0,NaN,"150,000km",NaN,volkswagen,limousine,golf,nein
freq,49999,1421,NaN,32424,NaN,10687,12859,4024,35232
mean,NaN,NaN,116.355920,NaN,2005.073280,NaN,NaN,NaN,NaN
std,NaN,NaN,209.216627,NaN,105.712813,NaN,NaN,NaN,NaN
min,NaN,NaN,0.000000,NaN,1000.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,70.000000,NaN,1999.000000,NaN,NaN,NaN,NaN
50%,NaN,NaN,105.000000,NaN,2003.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,150.000000,NaN,2008.000000,NaN,NaN,NaN,NaN


Some interesting findings:
- *seller* column only have two unique values, and one of them have a frequency of 49999. This frequency is almost the same as the total count, and therefore analyzing these two columns may not give me any useful information. 
- *unrepaired_damage* column has almost all values being "nein".

Therefore, selecting rows with only private seller and no unrepaired_damage can still give me enough data to build the model.

In [7]:
# Select only private seller and no unrepaired_damage, and remove the two columns
autos = autos.loc[(autos['seller']=='privat') & (autos['unrepaired_damage']=='nein'),:]
autos.drop(columns=['seller','unrepaired_damage'],inplace=True)

Since my goal is to determine how do power and milage impact price in three most common brands, I can remove *model* column as it is brand-specific. The vehicle type is also not in the scope of my analysis, so I will remove it too.

In [8]:
# Drop "model" and "vehicle_type" columns
autos.drop(columns=['model','vehicle_type'], inplace=True)

The new dataframe is as follow.

In [9]:
autos.describe(include='all')

,price,power_ps,odometer,registration_year,brand
count,35232,35232.000000,35232,35232.000000,35232
unique,2184,NaN,13,NaN,40
top,"$1,500",NaN,"150,000km",NaN,volkswagen
freq,453,NaN,21352,NaN,7407
mean,NaN,126.560911,NaN,2004.392172,NaN
std,NaN,202.017709,NaN,46.863260,NaN
min,NaN,0.000000,NaN,1800.000000,NaN
25%,NaN,75.000000,NaN,2000.000000,NaN
50%,NaN,116.000000,NaN,2004.000000,NaN
75%,NaN,150.000000,NaN,2009.000000,NaN


Notice that *price* and *odometer* columns are in "object" data type. I need to convert them into numerical type.

In [10]:
# Change the data type of columns "price" and "odometer" into float
autos['price'] = autos['price'].str.replace('$','')
autos['price'] = autos['price'].str.replace(',','')
autos['price'] = autos['price'].astype(float)


autos['odometer'] = autos['odometer'].str.replace('km','')
autos['odometer'] = autos['odometer'].str.replace(',','')
autos['odometer'] = autos['odometer'].astype(float)

# Rename the column titles to make them descriptive
autos.rename(columns={'price':'price_dollar', 'odometer':'odometer_km'},inplace=True)

In [11]:
autos['registration_year'] = autos['registration_year'].astype(float)
autos['power_ps'] = autos['power_ps'].astype(float)
autos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35232 entries, 0 to 49999
Data columns (total 5 columns):
price_dollar         35232 non-null float64
power_ps             35232 non-null float64
odometer_km          35232 non-null float64
registration_year    35232 non-null float64
brand                35232 non-null object
dtypes: float64(4), object(1)
memory usage: 1.6+ MB


Now, let's investigate columns *price_dollar* and *odometer_km* to see if additional cleaning is required.

In [12]:
autos['price_dollar'].describe()

count    3.523200e+04
mean     8.470596e+03
std      1.248084e+05
min      0.000000e+00
25%      1.700000e+03
50%      3.999000e+03
75%      8.950000e+03
max      1.234568e+07
Name: price_dollar, dtype: float64

It is noticed that the min price is 0 dollar, and the max price is 1,234,568 dollars. It is not normal to sell a car with a price of 0 dollar or higher than 500,000 dollars (the most luxurious cars in the world, like [Rolls Royce Phantom](https://www.rolls-roycemotorcars.com/en_GB/showroom/phantom.html) have price of 500,000 dollars. and the price is for brand new cars!). So, I need to set a proper price range for my analysis.

First, let's find the [interquartile range](https://en.wikipedia.org/wiki/Interquartile_range) (the difference between the third quartile and the first quartile). Based on the description above, the first quartile is 1,700 dollars, and the third quartile is 8,950 dollars.

In [13]:
# Find the interquartile range
interq_price = 8950 - 1700

The [major outliers](https://en.wikipedia.org/wiki/Outlier) are outside of the "outer fences". The upper bound of the "outer fences" is three interquartile range above the third quartile; the lower bound of the "outer fences" is three interquartile range below the first quartile.

In [14]:
# Find the outer fences
up_price = 8950 + 3 * interq_price
low_price = 1700 - 3 * interq_price
print("The upper bound is {num}".format(num=up_price))
print("The lower bound is {num}".format(num=low_price))

The upper bound is 30700
The lower bound is -20050


The lower bound of the "outer fences" does not make sense as it is a negative value. However, essentially it indicates that all values below the first quartile is not considered outlier. Still, based on the common sense, nobody should sell a call with 0 dollar. So I take all values between 0 dollar and 25500 dollars, excluding 0 dollar for my analysis.

In [15]:
autos = autos.loc[autos['price_dollar'].between(0.1, 30700)]

Now, let's examine the odometer_km column.

In [16]:
autos['odometer_km'].describe()

count     34060.000000
mean     124396.359366
std       39339.484372
min        5000.000000
25%      100000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

The values in *odometer_km* column look reasonable. However, it is oberserved that the median value is equal to the max value. This indicates that over half of cars have odometer equal to 150,000 km. It is rare to see most of the cars having exactly the same odometer. I can think of two potential explainations: 
1. The cars with higher than 150,000 km are not included in the data set.
2. The odometers higher than 150,000 km are considered as 150,000 km.

In either way, considering that more than half of the data in this column is the same, this column may not be very useful for our analysis.

One alternative for odometer is the registration year. In general, the older the car, the higher the milage. So let's have a look at the *registration_year* column.

In [17]:
autos['registration_year'].describe()

count    34060.000000
mean      2004.005608
std         17.720396
min       1800.000000
25%       2000.000000
50%       2004.000000
75%       2008.000000
max       5000.000000
Name: registration_year, dtype: float64

Obviously, the min and max for this column are wrong. So i will use the "outer fences" to filter out the outliers.

In [18]:
# Filter out the outliers in "registration_year" column
interq_reg = 2008 - 2000
up_reg = 2008 + 3 * interq_reg
low_reg = 2000 - 3 * interq_reg
print("The upper bound is {num}".format(num=up_reg))
print("The lower bound is {num}".format(num=low_reg))

The upper bound is 2032
The lower bound is 1976


The lower bound found by "outer fences" method is reasonable. However, the upper bound is not. Thus, to find the correct upper bound, I need to further check the column.

In [19]:
autos['registration_year'].value_counts().sort_index(ascending=False).head(10)

5000.0       1
2019.0       2
2018.0     260
2017.0     672
2016.0     490
2015.0     243
2014.0     501
2013.0     659
2012.0    1088
2011.0    1396
Name: registration_year, dtype: int64

It can be noticed that the latest year should be 2019. So the final range is from 1976 to 2019

In [20]:
autos = autos.loc[autos['registration_year'].between(1976,2019)]

Another parameter I need to fit into my model is the power of the car. So let's check *power_ps* column.

In [21]:
autos['power_ps'].describe()

count    33856.000000
mean       123.666322
std        203.376915
min          0.000000
25%         75.000000
50%        115.000000
75%        150.000000
max      17700.000000
Name: power_ps, dtype: float64

Apparently, a power of 17700 or 0 does not make sense. Again, let's use the "outer fences" method first to clean our data.

In [22]:
# Filter out the outliers in "power_ps" column
interq_power = 150 - 75
up_power = 150 + 3 * interq_power
low_power = 75 - 3 * interq_power
print("The upper bound is {num}".format(num=up_power))
print("The lower bound is {num}".format(num=low_power))

The upper bound is 375
The lower bound is -150


A negative power is not physically possible. From a quick web search, the lowest hoursepower 

In [23]:
autos['power_ps'].value_counts().sort_index(ascending=True).head(10)

0.0     1812
1.0        2
2.0        1
3.0        1
4.0        4
5.0       10
6.0        1
8.0        1
10.0       2
11.0       4
Name: power_ps, dtype: int64

Suprisingly, there are many cars with power less than 10. It's quite rare to see any cars with such low power today, so my first impression is that they are very old cars. Let's verify if it is truely the case.

In [24]:
autos.loc[autos['power_ps']==1]

,price_dollar,power_ps,odometer_km,registration_year,brand
485,490.0,1.0,100000.0,2002.0,chrysler
47530,5900.0,1.0,125000.0,2017.0,bmw


The chart shows that a 2017 BMW has a power of 1 PS. This does not make sense as [the least powerful car](https://www.autoguide.com/auto-news/2017/07/top-10-cars-with-the-least-horsepower-in-2017.html) in 2017 that's available today has a power of 78 HP (79 PS). So I consider 79 PS as my lower bound for power. 

In [25]:
autos = autos.loc[autos['power_ps'].between(79,359)]

Now, I need to find the top three most famous brands for this data set. 

In [26]:
autos['brand'].value_counts().head(3)

volkswagen       4746
bmw              3732
mercedes_benz    3078
Name: brand, dtype: int64

**The top three brands are Volkswagen, BMW, Mercedes_benz.** 

## Model Building

Let's use the data of the top three brands to build a linear regression model that predicts the prices of the used cars!

In [27]:
autos_vol = autos.loc[autos['brand']=='volkswagen']
autos_bmw = autos.loc[autos['brand']=='bmw']
autos_benz = autos.loc[autos['brand']=='mercedes_benz']

The following code uses multiple regression model in sklearn to build the model. 

In [28]:
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

X1 = autos_vol.loc[:,['power_ps', 'registration_year']]
y1 = autos_vol.loc[:,'price_dollar']

X1.loc[:,['power_ps', 'registration_year']] = scale.fit_transform(X1.loc[:,['power_ps', 'registration_year']].as_matrix())
X1 = sm.add_constant(X1)

est_vol = sm.OLS(y1, X1).fit()

est_vol.summary()

/dataquest/system/env/python3/lib/python3.4/importlib/_bootstrap.py:321: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  return f(*args, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           price_dollar   R-squared:                       0.522
Model:                            OLS   Adj. R-squared:                  0.522
Method:                 Least Squares   F-statistic:                     2587.
Date:                Mon, 30 Mar 2020   Prob (F-statistic):               0.00
Time:                        15:16:34   Log-Likelihood:                -46480.
No. Observations:                4746   AIC:                         9.297e+04
Df Residuals:                    4743   BIC:                         9.299e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const              8010.0504     62.953    127.238      0.000    7886.632    8133.468
power_ps           2050.0285     64.412     31.827      0.000    1923.751    2176.306
registration_year  3627.2536     64.412     56.313      0.000    3500.976    3753.531
==============================================================================
Omnibus:                      767.003   Durbin-Watson:                   1.982
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2217.999
Skew:                           0.852   Prob(JB):                         0.00
Kurtosis:                       5.883   Cond. No.                         1.24
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [29]:
X2 = autos_bmw.loc[:,['power_ps', 'registration_year']]
y2 = autos_bmw.loc[:,'price_dollar']

X2.loc[:,['power_ps', 'registration_year']] = scale.fit_transform(X2.loc[:,['power_ps', 'registration_year']].as_matrix())
X2 = sm.add_constant(X2)

est_bmw = sm.OLS(y2, X2).fit()

est_bmw.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           price_dollar   R-squared:                       0.541
Model:                            OLS   Adj. R-squared:                  0.541
Method:                 Least Squares   F-statistic:                     2199.
Date:                Mon, 30 Mar 2020   Prob (F-statistic):               0.00
Time:                        15:16:34   Log-Likelihood:                -36658.
No. Observations:                3732   AIC:                         7.332e+04
Df Residuals:                    3729   BIC:                         7.334e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const              8367.7840     73.100    114.471      0.000    8224.464    8511.104
power_ps           2458.4510     74.587     32.961      0.000    2312.217    2604.685
registration_year  3718.6676     74.587     49.857      0.000    3572.433    3864.902
==============================================================================
Omnibus:                      419.319   Durbin-Watson:                   1.969
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1289.130
Skew:                           0.583   Prob(JB):                    1.17e-280
Kurtosis:                       5.633   Cond. No.                         1.22
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [30]:
X3 = autos_benz.loc[:,['power_ps', 'registration_year']]
y3 = autos_benz.loc[:,'price_dollar']

X3.loc[:,['power_ps', 'registration_year']] = scale.fit_transform(X3.loc[:,['power_ps', 'registration_year']].as_matrix())
X3 = sm.add_constant(X3)

est_benz = sm.OLS(y3, X3).fit()

est_benz.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           price_dollar   R-squared:                       0.443
Model:                            OLS   Adj. R-squared:                  0.443
Method:                 Least Squares   F-statistic:                     1224.
Date:                Mon, 30 Mar 2020   Prob (F-statistic):               0.00
Time:                        15:16:35   Log-Likelihood:                -30759.
No. Observations:                3078   AIC:                         6.152e+04
Df Residuals:                    3075   BIC:                         6.154e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const              8301.4370     95.447     86.974      0.000    8114.291    8488.583
power_ps           2745.1975     96.333     28.497      0.000    2556.315    2934.080
registration_year  3488.7200     96.333     36.215      0.000    3299.837    3677.603
==============================================================================
Omnibus:                      589.896   Durbin-Watson:                   1.970
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1377.093
Skew:                           1.072   Prob(JB):                    9.29e-300
Kurtosis:                       5.478   Cond. No.                         1.15
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [37]:
# Compare the prices of three brands with 120 PS and registration year of 2018
car = [120, 2018]    # Car with 120 PS and registration year of 2018
scaled = scale.transform([car])   # Standardization the values

scaled= sm.add_constant(scaled, has_constant='add')
predicted_vol = est_vol.predict(scaled)
predicted_bmw = est_bmw.predict(scaled)
predicted_benz = est_benz.predict(scaled)

print("The predicted price for Volkswagen is {num} dollars".format(num = int(predicted_vol)))
print("The predicted price for BMW is {num} dollars".format(num = int(predicted_bmw)))
print("The predicted price for Benz is {num} dollars".format(num = int(predicted_benz)))

The predicted price for Volkswagen is 14714
The predicted price for BMW is 14957
The predicted price for Benz is 14133


From the predictive model it can be found that:
- Both the registration year and the power have positive correlation with price.
- The registration year has larger impact on price as compared to the power.
- All three brands have similar model coefficients on registration year.
- Volkswagen has a slightly lower base price, indicated by a lower constant coefficient
- Volkswagen has significantly lower coefficient value on the power. This means that the power does not impact the price of Volswagen used cars as much as BMW and Benz used cars.
- For a 2018 used car with 120 PS ([average car power](https://www.autolist.com/guides/average-car-horsepower)), the model predicts that Benz brand has the lowest price at 14133 dollars, BMW has the highest price at 14957 dollars. 

# Conclusion
In summary, the top three common brands of used cars on *eBay Kleinanzeigen* are Volkswage, BMW and Benz. Predicted by my model, for a 2018 used car with 120 PS power, Benz brand has the lowest price at 14133 dollars. Thus, if you are looking for a 2018 used car, I recommend checking Benz first.